In [ ]:
!pip install matplotlib
import tensorflow as tf
from tensorflow.keras import models, layers
import matplotlib.pyplot as plt

In [ ]:
BATCH_SIZE = 30
IMAGE_SIZE = 224
CHANNELS=3
EPOCHS=20

In [ ]:
dataset = tf.keras.preprocessing.image_dataset_from_directory(
    "skin-disease-datasaet",
    seed=123,
    shuffle=True,
    image_size=(IMAGE_SIZE,IMAGE_SIZE),
    batch_size=BATCH_SIZE
)

In [ ]:
class_names = dataset.class_names
class_names

In [ ]:
for image_batch, labels_batch in dataset.take(1):
    print(image_batch.shape)
    print(labels_batch.numpy())


In [ ]:
train_size = 0.8
len(dataset)*train_size

In [ ]:
train_ds = dataset.take(30)
len(train_ds)

In [ ]:
test_ds = dataset.skip(30)
len(test_ds)


In [ ]:
val_size=0.1
len(dataset)*val_size

In [ ]:
val_ds = test_ds.take(3)
len(val_ds)


In [ ]:
test_ds = test_ds.skip(3)
len(test_ds)

In [ ]:
def get_dataset_partitions_tf(ds, train_split=0.8, val_split=0.1, test_split=0.1, shuffle=True, shuffle_size=10000):
    assert (train_split + test_split + val_split) == 1
    
    ds_size = len(ds)
    
    if shuffle:
        ds = ds.shuffle(shuffle_size, seed=12)
    
    train_size = int(train_split * ds_size)
    val_size = int(val_split * ds_size)
    
    train_ds = ds.take(train_size)    
    val_ds = ds.skip(train_size).take(val_size)
    test_ds = ds.skip(train_size).skip(val_size)
    
    return train_ds, val_ds, test_ds


In [ ]:
train_ds, val_ds, test_ds = get_dataset_partitions_tf(dataset)


In [ ]:
resize_and_rescale = tf.keras.Sequential([
  layers.experimental.preprocessing.Resizing(IMAGE_SIZE, IMAGE_SIZE),
  layers.experimental.preprocessing.Rescaling(1./255),
])


In [ ]:
input_shape = (BATCH_SIZE, IMAGE_SIZE, IMAGE_SIZE, CHANNELS)
n_classes = 8

model = models.Sequential([
    resize_and_rescale,
    layers.Conv2D(32, kernel_size = (3,3), activation='relu', input_shape=input_shape),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64,  kernel_size = (3,3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64,  kernel_size = (3,3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(n_classes, activation='softmax'),
])

model.build(input_shape=input_shape)


In [ ]:
model.summary()

In [ ]:
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)


In [ ]:
history = model.fit(
    train_ds,
    batch_size=BATCH_SIZE,
    validation_data=val_ds,
    verbose=1,
    epochs=20,
)

In [ ]:
scores = model.evaluate(test_ds)

In [ ]:
scores

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']


In [ ]:
import tensorflow as tf
from tensorflow.keras import models, layers
from sklearn.metrics import classification_report

# ... (Your previous code)

# Train the model
history = model.fit(
    train_ds,
    batch_size=BATCH_SIZE,
    validation_data=val_ds,
    verbose=1,
    epochs=10,
)

# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(test_ds, verbose=2)
print(f'Test accuracy: {test_acc}')

# Predict labels for the test set
predictions = model.predict(test_ds)

# Convert predictions to class labels
predicted_labels = tf.argmax(predictions, axis=1)

# Get true labels from the test set
true_labels = tf.concat([y for x, y in test_ds], axis=0)

# Convert to numpy arrays for classification_report
predicted_labels_np = predicted_labels.numpy()
true_labels_np = true_labels.numpy()

# Generate and print classification report
class_report = classification_report(true_labels_np, predicted_labels_np, target_names=class_names)
print("Classification Report:\n", class_report)


In [ ]:
def predict(model, img):
    img_array = tf.keras.preprocessing.image.img_to_array(images[i].numpy())
    img_array = tf.expand_dims(img_array, 0)
    predictions = model.predict(img_array)
    predicted_class = class_names[np.argmax(predictions[0])]
    confidence = round(100 * (np.max(predictions[0])), 2)
    return predicted_class, confidence

In [ ]:
import numpy as np 
plt.figure(figsize=(15, 15))
for images, labels in test_ds.take(1):
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        
        predicted_class, confidence = predict(model, images[i].numpy())
        actual_class = class_names[labels[i]] 
        
        plt.title(f"Actual: {actual_class},\n Predicted: {predicted_class}.\n Confidence: {confidence}%")
        
        plt.axis("off")

       

In [ ]:
import numpy as np
for images_batch, labels_batch in test_ds.take(1):
    
    first_image = images_batch[0].numpy().astype('uint8')
    first_label = labels_batch[0].numpy()
    
    print("first image to predict")
    plt.imshow(first_image)
    print("actual label:",class_names[first_label])
    
    batch_prediction = model.predict(images_batch)
    print("predicted label:",class_names[np.argmax(batch_prediction[0])])


In [ ]:
import os
directory_path = "../models3"

# Check if the directory exists, if not, create it
if not os.path.exists(directory_path):
    os.makedirs(directory_path)

model_version=max([int(i) for i in os.listdir("../models3") + [0]])+1
model.save(f"models3/{model_version}")

In [ ]:
model.save("../skin.h5")

In [ ]:
import tensorflow as tf
from tensorflow.keras import models
from sklearn.metrics import classification_report

# Assuming you have already trained your model and saved it to an h5 file
# If you haven't saved it yet, you can do so using: model.save("your_model.h5")

# Load the saved model from the h5 file
loaded_model = models.load_model("my_.h5")

# Evaluate the model on the test set
test_loss, test_acc = loaded_model.evaluate(train_ds, verbose=2)
print(f'Test accuracy: {test_acc}')

# Predict labels for the test set
predictions = loaded_model.predict(train_ds)

# Convert predictions to class labels
predicted_labels = tf.argmax(predictions, axis=1)

# Get true labels from the test set
true_labels = tf.concat([y for x, y in train_ds], axis=0)

# Convert to numpy arrays for classification_report
predicted_labels_np = predicted_labels.numpy()
true_labels_np = true_labels.numpy()
A
# Generate and print classification report
class_report = classification_report(true_labels_np, predicted_labels_np, target_names=class_names)
print("Classification Report:\n", class_report)
